# **yolov5로 이미지 객체 인식 모델 만들기**

원본 동영상 강의 https://www.youtube.com/watch?v=T0DO1C8uYP8&t=122s


## **객체 인식모델 구축 전 준비사항**
**제공된 dataset 폴더를 구글 드라이브에 넣어두기**


## **YOLOv5 다운로드 및 환경설정**

In [2]:
%cd /content
!git clone https://github.com/ultralytics/yolov5.git

/content
Cloning into 'yolov5'...
remote: Enumerating objects: 7462, done.
remote: Counting objects: 100% (130/130), done.
remote: Compressing objects: 100% (88/88), done.
remote: Total 7462 (delta 82), reused 80 (delta 42), pack-reused 7332
Receiving objects: 100% (7462/7462), 9.00 MiB | 24.79 MiB/s, done.
Resolving deltas: 100% (5134/5134), done.


In [ ]:
%cd /content/yolov5/
!pip install -r requirements.txt

/content/yolov5


##**구글 드라이브 마운트**

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## **data.yaml : 데이터 셋의 정보를 담은 파일**

> 분류할 객체이름 : Bollard

> 객체 갯수 : 1 

> train data의 이름을 명시한 텍스트 파일

> valid data의 이름을 명시한 텍스트 파일










In [ ]:
%cat /content/drive/MyDrive/dataset/data.yaml

names:
- Bollard
nc: 1
train: /content/drive/MyDrive/dataset/train.txt
val: /content/drive/MyDrive/dataset/val.txt


## **학습시킬 이미지를 img_list에 추가**

In [ ]:
from glob import glob

# path : 학습할 이미지가 저장된 경로 exts : 이미지 파일 유형
def getfiles(path,exts):
    files = []
    for e in exts:
        for f in glob(path + "*."+e, recursive=True):
            files.append(f)
    return files

path="/content/drive/MyDrive/dataset/img/"
exts =["JPG","jpg"]
img_list=getfiles(path,exts)
print(len(img_list))

59


### **-> 실제 학습에 사용한 이미지는 563장**

In [ ]:
from glob import glob

def getfiles(path,exts):
    files = []
    for e in exts:
        for f in glob(path + "*."+e, recursive=True):
            files.append(f)
    return files

path="/content/drive/MyDrive/4-1_종합설계1_플랫로드/labelling/"
exts =["JPG","jpg"]
img_list=getfiles(path,exts)
print(len(img_list))

563


## **train data와 test data 구축하기**

**전체이미지를 train data와 test data로 랜덤하게 나누기**

In [ ]:
from sklearn.model_selection import train_test_split

train_img_list, val_img_list = train_test_split(img_list, test_size=0.2, random_state=2000)

print(len(train_img_list), len(val_img_list))

450 113


**train data와 test data의 경로를 저장하고 있는 텍스트 파일 생성**

In [ ]:
with open('/content/drive/MyDrive/dataset/train.txt', 'w') as f:
  f.write('\n'.join(train_img_list) + '\n')

with open('/content/drive/MyDrive/dataset/val.txt', 'w') as f:
  f.write('\n'.join(val_img_list) + '\n')

**data.yaml 파일의 train data와 test data 경로를 수정**

In [ ]:
import yaml

with open('/content/drive/MyDrive/dataset/data.yaml', 'r') as f:
  data = yaml.load(f)

print(data)

data['train'] = '/content/drive/MyDrive/dataset/train.txt'
data['val'] = '/content/drive/MyDrive/dataset/val.txt'

with open('/content/drive/MyDrive/dataset/data.yaml', 'w') as f:
  yaml.dump(data, f)

print(data)

{'names': ['Bollard'], 'nc': 1, 'train': '/content/drive/MyDrive/dataset/train.txt', 'val': '/content/drive/MyDrive/dataset/val.txt'}
{'names': ['Bollard'], 'nc': 1, 'train': '/content/drive/MyDrive/dataset/train.txt', 'val': '/content/drive/MyDrive/dataset/val.txt'}


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  after removing the cwd from sys.path.


# **이미지 데이터로 yolov5s 모델을 학습** : 50 epoch를 학습하도록 설정
학습된 모델은 /yolov5/runs/bollard_yolov5s_result에 저장

In [ ]:
%cd /content/yolov5/

!python train.py --img 416 --batch 16 --epochs 50 --data /content/drive/MyDrive/dataset/data.yaml --cfg ./models/yolov5s.yaml --weights yolov5s.pt --name bollard_yolov5s_results

/content/yolov5
train: weights=yolov5s.pt, cfg=./models/yolov5s.yaml, data=/content/drive/MyDrive/dataset/data.yaml, hyp=data/hyp.scratch.yaml, epochs=50, batch_size=16, img_size=[416], rect=False, resume=False, nosave=False, notest=False, noautoanchor=False, evolve=False, bucket=, cache_images=False, image_weights=False, device=, multi_scale=False, single_cls=False, adam=False, sync_bn=False, workers=8, project=runs/train, entity=None, name=bollard_yolov5s_results, exist_ok=False, quad=False, linear_lr=False, label_smoothing=0.0, upload_dataset=False, bbox_interval=-1, save_period=-1, artifact_alias=latest, local_rank=-1
github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 v5.0-223-gb83e1a4 torch 1.9.0+cu102 CPU

hyperparameters: lr0=0.01, lrf=0.2, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.0, obj_pw=1.0, iou_t=0.2, anchor_t=4.0, fl_gamma=0.0, hsv_h=0.015, hsv_s=0.7, hsv_v=0.

## **학습된 모델이 볼라드를 잘 인식하는지 확인하기**
### **확인된 이미지는 yolov5/runs/dect 밑에 추가된다.**

In [4]:
!python /content/yolov5/detect.py --source /content/drive/MyDrive/4-1_종합설계1_플랫로드/imgtrain/fheldd.JPG --weights /content/drive/MyDrive/4-1_종합설계1_플랫로드/imgtrain/best.pt

detect: weights=['/content/drive/MyDrive/4-1_종합설계1_플랫로드/imgtrain/best.pt'], source=/content/drive/MyDrive/4-1_종합설계1_플랫로드/imgtrain/fheldd.JPG, imgsz=640, conf_thres=0.25, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False
requirements: /content/requirements.txt not found, check failed.
YOLOv5 🚀 v5.0-235-gffb6e11 torch 1.9.0+cu102 CPU

Fusing layers... 
Model Summary: 224 layers, 7053910 parameters, 0 gradients
/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.